In [1]:
# SymPT imports
from SymPT import *
# import sympy
import sympy as sp

# Setup

In [2]:
# ---------------- Defining the symbols ------------------
# Order 0
omega = RDSymbol('omega', positive=True, real=True)
omega_z = RDSymbol('omega_z', positive=True, real=True)
omega_d = RDSymbol('omega_d', positive=True, real=True)

# Order 1
bsl = RDSymbol('\\tilde{b}_{SL}', order=1, positive=True, real=True)
E0 = RDSymbol('\\tilde{E}_{0}', order=1, positive=True, real=True)

# ----------------- Defining the basis -------------------
# Spin basis: Finite 2x2 Hilbert space
Spin = RDBasis('sigma', dim=2)
s0, sx, sy, sz = Spin.basis

# Boson basis: Infinite bosonic Hilbert space
a = BosonOp('a')
ad = Dagger(a)


# -------------- Defining the Hamiltonian ----------------
# Unperturbed Hamiltonian H0
H0 = hbar * omega * ad*a - hbar/2 * omega_z * sz

# Perturbation Hamiltonians
V = -sp.Rational(1,2) * hbar * bsl * (ad + a) * sx
Hd = -E0 * sp.cos(omega_d * t) * (ad + a)

display_dict({
    sp.Symbol('H_0'): H0,
    sp.Symbol('V'):   V,
    sp.Symbol('H_d'): Hd
})

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

# Time Independent SWT

In [3]:
# Deffining Effective Hamiltonian Object
Eff_frame = EffectiveFrame(H0, V, subspaces=[Spin])

The EffectiveFrame object has been initialized successfully.


In [4]:
# Calculate the effective model using the Schrieffer-Wolff transformation up to the second order
Eff_frame.solve(max_order=2, method="SW", mask=None)
# Obtaining the result in the operator form
H_eff_SWT = Eff_frame.get_H(return_form='operator')
H_eff_SWT.cancel().expand()

Performing SWT for each order: 100%|██████████| 2/2 [00:00<00:00, 17.40it/s]
The Hamiltonian has been solved successfully. Please use the get_H method to get the result in the desired form.
Converting to operator form: 100%|██████████| 2/2 [00:00<00:00, 15.94it/s]


-\tilde{b}_{SL}**2*hbar*omega/(4*omega**2 - 4*omega_z**2) + \tilde{b}_{SL}**2*hbar*omega_z*sigma_3*Dagger(a)**2/(4*omega**2 - 4*omega_z**2) + \tilde{b}_{SL}**2*hbar*omega_z*sigma_3*a**2/(4*omega**2 - 4*omega_z**2) + \tilde{b}_{SL}**2*hbar*omega_z*sigma_3*Dagger(a)*a/(2*omega**2 - 2*omega_z**2) - \tilde{b}_{SL}**2*hbar*sigma_3/(8*omega + 8*omega_z) + \tilde{b}_{SL}**2*hbar*sigma_3/(8*omega - 8*omega_z) + hbar*omega*Dagger(a)*a - hbar*omega_z*sigma_3/2

In [5]:
Heff_2_dict = group_by_operators(Eff_frame.corrections[2])
(Heff_2_dict[sz].cancel() + Heff_2_dict[sz*ad**2].cancel() * ad**2 + Heff_2_dict[sz*a**2].cancel() * a**2 + Heff_2_dict[sz*ad*a].cancel()*ad*a).cancel() * sz

\tilde{b}_{SL}**2*hbar*omega_z*(1 + 2*Dagger(a)*a + Dagger(a)**2 + a**2)*sigma_3/(4*omega**2 - 4*omega_z**2)

# Time Dependent SWT

In [6]:
# Deffining Effective Hamiltonian Object
Eff_frame = EffectiveFrame(H0, V + Hd, subspaces=[Spin])

The EffectiveFrame object has been initialized successfully.


In [7]:
# Calculate the effective model using the Schrieffer-Wolff transformation up to the second order
Eff_frame.solve(max_order=2, method='SW')
# Obtaining the result in the operator form
H_eff_SWT = Eff_frame.get_H(return_form='operator')

Performing Time Dependent SWT for each order: 100%|██████████| 2/2 [00:00<00:00, 10.48it/s]
The Hamiltonian has been solved successfully. Please use the get_H method to get the result in the desired form.
Converting to operator form: 100%|██████████| 2/2 [00:23<00:00, 11.92s/it]


In [8]:
H2_Teff = Eff_frame.corrections[2]

In [9]:
H2_Teff_dict = group_by_operators(H2_Teff)
print(H2_Teff_dict.keys())
(H2_Teff_dict[sz].cancel() + H2_Teff_dict[sz*ad**2].cancel() * ad**2 + H2_Teff_dict[sz*a**2].cancel() * a**2 + H2_Teff_dict[sz*ad*a].cancel()*ad*a).cancel() * sz

dict_keys([1, sigma_3, sigma_1, sigma_3*Dagger(a)**2, sigma_3*a**2, sigma_3*Dagger(a)*a])


\tilde{b}_{SL}**2*hbar*omega_z*(1 + 2*Dagger(a)*a + Dagger(a)**2 + a**2)*sigma_3/(4*omega**2 - 4*omega_z**2)

In [10]:
sp.expand_complex((H2_Teff_dict[sx]).cancel()).trigsimp().factor()

-\tilde{E}_{0}*\tilde{b}_{SL}*omega*(2*omega**2 - omega_d**2 - omega_z**2)*cos(omega_d*t)/(2*(omega - omega_d)*(omega + omega_d)*(omega - omega_z)*(omega + omega_z))